<a href="https://colab.research.google.com/github/giicis/MetricsEvaluationOptimizationMethodsDynamicProblems/blob/main/VRP_Dynamic_ACO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#    Función: Calcula la longitud total del recorrido en el TSP, es decir, la distancia total que se recorre al visitar todas las ciudades y regresar a la ciudad inicial.

#input:
    #tour: Una lista que representa un recorrido de ciudades. Cada elemento de la lista es un índice que representa una ciudad en el modelo.
    #model: Un diccionario que contiene información sobre el problema TSP, incluyendo las coordenadas de las ciudades y una matriz de distancias entre ellas.


#Output: Retorna la longitud total del recorrido.
def tour_length(tour, model):
    n = len(tour)  # Obtiene el número de ciudades en el recorrido

  # Añade la primera ciudad al final para completar el recorrido circular
    tour.append(tour[0])

    L = 0  # Inicializa la longitud del recorrido

    # Itera sobre el recorrido
    for i in range(n):
        # Accede a la matriz de distancias del modelo y acumula la longitud del recorrido
        # Se resta 1 para ajustar el índice, ya que Python usa índices basados en 0
        L += model.get_dist()[tour[i], tour[i+1]]

    return L  # Retorna la longitud total del recorrido


#    Función: Devuelve la lista que contiene los distintos sub-tour de la solucion.
#input:
    #tour: Una lista que representa un recorrido de ciudades. Cada elemento de la lista es un índice que representa una ciudad en el modelo.
    #sep: El indice que corresponde al deposito. Sirve para separar un sub-tour de otro.


#Output: Retorna una lista con los distintos sub-tours (cada uno de ellos una lista a su vez)

def sub_touring(tour, sep):
    last = len(tour) - 1
    if last == 0: return [tour] #Si recien empieza, se devuelve el nodo inicial
    result = []
    for i, v in enumerate(tour):
        if v == sep:
            result.append([])
            result[-1].append(sep)
        else:
            result[-1].append(v)
    return result


**El propósito de la función roulette_wheel_selection(P)** es seleccionar un índice de un arreglo P de probabilidades dado. imita el proceso de selección en una ruleta de casino. En una ruleta, los sectores más grandes (que representan eventos más probables) tienen una mayor área y, por lo tanto, una mayor probabilidad de ser seleccionados cuando la ruleta gira.

Supongamos que tenemos un arreglo P que representa las probabilidades de seleccionar diferentes elementos. Esta función ayuda a elegir un índice de manera que los elementos con mayores probabilidades tengan una mayor probabilidad de ser seleccionados.

Aquí hay un ejemplo simplificado: supongamos que P es [0.2, 0.3, 0.1, 0.4]. Esto significa que el primer elemento tiene una probabilidad del 20%, el segundo del 30%, el tercero del 10% y el cuarto del 40%.

La función roulette_wheel_selection(P) generará un número aleatorio entre 0 y 1 (por ejemplo, 0.25). Luego, calculará la suma acumulativa de las probabilidades en P, que sería [0.2, 0.5, 0.6, 1.0]. A continuación, buscará el primer índice donde el número aleatorio (0.25) sea menor o igual a la suma acumulativa. En este caso, sería el segundo índice, lo que significa que el elemento correspondiente en P (en este caso, el segundo elemento) será seleccionado.

Esta selección basada en probabilidades es fundamental en ACO, ya que permite a las hormigas tomar decisiones sobre qué acción tomar a continuación, equilibrando entre explorar nuevas posibilidades y explotar las mejores soluciones conocidas.

In [ ]:
import random  # genera números aleatorios
import numpy as np

#Funcion: Implementa la selección de la ruleta para elegir un índice basado en probabilidades.
#Input: Un array P de probabilidades.
#Output: se devuelve el índice j, que será utilizado para seleccionar un elemento basado en las probabilidades proporcionadas en P.

def roulette_wheel_selection(P):
    r = random.random()  # Genera un número de punto flotante aleatorio entre 0 y 1
    C = np.cumsum(P)  # Calcula la suma acumulativa de los elementos en el arreglo 'P'
    j = np.argmax(r <= C)  # Encuentra el primer índice donde r es menor o igual que 'C'
    return j


#Esta función es fundamental para el algoritmo ACO porque permite a las hormigas tomar decisiones sobre qué ciudad visitar a continuación.
#La selección de la siguiente ciudad se basa en las feromonas depositadas y en la información heurística (en el cálculo de P), lo que equilibra la exploración y la explotación del espacio de búsqueda.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

#Funcion: Grafica la solucion

#Input:
#     tour: Una lista que representa un recorrido de ciudades. Cada elemento es un índice que representa una ciudad en el modelo.
#     model: Un diccionario que contiene información sobre el problema TSP, incluyendo las coordenadas de las ciudades y una matriz de distancias entre ellas.
#     name: Un nombre para el gráfico.

#Output: Grafico
def plot_solution(tour, model, name):

  subtours = []
  aux = []
  for node in tour:
    if node == 0:
      if len(aux) >= 1:
        aux.insert(0, 0)
        aux.append(0)
        subtours.append(aux)
        aux = []
    else:
      aux.append(node)

  colors = plt.cm.viridis(np.linspace(0, 1, len(subtours)))

  for sub, color in zip(subtours, colors):
   # Graficar el recorrido
    plt.plot(model.get_x()[sub], model.get_y()[sub], 'k-o',
         markersize=10, markerfacecolor='y', linewidth=1.5, color = color)

  # Etiquetas de los ejes y el título del gráfico
  plt.xlabel('x')
  plt.ylabel('y')
  plt.title(name)

  # Establecer el aspecto de los ejes como igual
  plt.axis('equal')

  # Habilitar la cuadrícula en el gráfico
  plt.grid(True)

  # Factor de ajuste
  alpha = 0.01

  # Ajustar los límites del eje x
  xmin = min(model.get_x())
  xmax = max(model.get_x())
  dx = xmax - xmin
  xmin = np.floor((xmin - alpha*dx)/10)*10
  xmax = np.ceil((xmax + alpha*dx)/10)*10
  plt.xlim([xmin, xmax])

  # Ajustar los límites del eje y
  ymin = min(model.get_y())
  ymax = max(model.get_y())
  dy = ymax - ymin
  ymin = np.floor((ymin - alpha*dy)/10)*10
  ymax = np.ceil((ymax + alpha*dy)/10)*10
  plt.ylim([ymin, ymax])


  plt.text(model.get_x()[tour[0]], model.get_y()[tour[0]], 'DEP',
           horizontalalignment='left', verticalalignment='bottom')
  num = 1
  for sub in subtours:
    for node in sub[1:-1]:
      t = f'P {num}'
      plt.text(model.get_x()[node], model.get_y()[node], t,
               horizontalalignment='left', verticalalignment='bottom')
      num += 1

  # Mostrar el gráfico
  plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


#Funcion: Grafica los niveles de feromonas.
#Input:
#      model: Un diccionario que contiene información sobre el problema TSP, incluyendo las coordenadas de las ciudades y una matriz de distancias entre ellas.
#      tau: Una matriz de niveles de feromonas.representa los niveles de feromonas entre las ciudades. Cada valor de tau[i, j] indica la cantidad de feromonas entre las ciudades i y j
#      name: Un nombre para el gráfico.

#Output: grafico.

def plot_pheromones(model, tau, name):
    # Crear una figura y ejes para el gráfico
    fig, ax = plt.subplots()

    # Determinar el valor máximo de tau para escalar el colormap
    max_tau = np.max(tau)

    # Crear un mapa de colores
    cmap = plt.get_cmap('viridis')  # Se puede cambiar 'viridis' por cualquier otro mapa de colores

    # Iterar sobre las ciudades del modelo
    for i in range(model.get_n()):
        x_0 = model.get_x()[i]  # Coordenada x de la ciudad i
        y_0 = model.get_y()[i]  # Coordenada y de la ciudad i

        # Iterar sobre las ciudades desde la ciudad i hasta la última
        for j in range(i, model.get_n()):
            if i != j:
                param = tau[i, j] / max_tau  # Normalizar tau para el mapa de colores (ajusta los valores de tau de manera que se encuentren dentro de un rango específico que sea adecuado para la representación gráfica en un colormap.)
                x_1 = model.get_x()[j]  # Coordenada x de la ciudad j
                y_1 = model.get_y()[j]  # Coordenada y de la ciudad j

                # Coordenadas de inicio y fin para la línea
                x = [x_0, x_1]
                y = [y_0, y_1]

                # Dibujar una línea con marcadores en los extremos
                ax.plot(x, y, '-', color=cmap(param), linewidth=2.5)

    # Establecer etiquetas y título
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(name)

    # Habilitar la cuadrícula en el gráfico
    ax.grid(True)

    # Ajustes para los límites de los ejes x e y
    alpha = 0.1
    xmin = min(model.get_x())
    xmax = max(model.get_x())
    dx = xmax - xmin
    xmin = np.floor((xmin - alpha*dx) / 10) * 10
    xmax = np.ceil((xmax + alpha*dx) / 10) * 10
    ax.set_xlim([xmin, xmax])

    ymin = min(model.get_y())
    ymax = max(model.get_y())
    dy = ymax - ymin
    ymin = np.floor((ymin - alpha*dy) / 10) * 10
    ymax = np.ceil((ymax + alpha*dy) / 10) * 10
    ax.set_ylim([ymin, ymax])

    # Agregar una barra de colores con el argumento 'ax'
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(0, max_tau))
    sm.set_array([])
    plt.colorbar(sm, ax=ax, label='Tau')

    # Mostrar la figura
    plt.show()


In [ ]:
import numpy as np

def city_distances(x, y):

  n = len(x)

  dist = np.zeros((n, n))

  for i in range(n-1):
        for j in range(i+1, n):
            dist[i, j] = np.sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2)
            dist[j, i] = dist[i, j]

  return dist

In [ ]:
import numpy as np

#Clase: Modelo que contiene la información de la iteración actual y sus modificaciones respecto de la anterior

class Model:

  def __init__(self, x, y):
    self.x = x
    self.y = y
    self.n = len(x)
    self.dist = city_distances(self.x, self.y)
    self.del_cities = {"n": 0,
                       "to_del": np.array([])}
    self.add_cities = {"n": 0}
    self.move_cities = {"n": 0,
                        "to_move": np.array([])}
    self.old_dist = np.array([])
    self.changed = False #Indica si el modelo actual cambió respecto al anterior.

  def get_x(self):
    return self.x

  def get_y(self):
    return self.y

  def get_n(self):
    return self.n

  def get_dist(self):
    return self.dist

  def get_cities_del(self):
    return self.del_cities["n"]

  def get_cities_to_del(self):
    return self.del_cities["to_del"]

  def get_cities_add(self):
    return self.add_cities["n"]

  def get_cities_move(self):
    return self.move_cities["n"]

  def get_cities_to_move(self):
    return self.move_cities["to_move"]

  def get_old_dist(self):
    return self.old_dist

  def has_changed(self):
    return self.changed

  def set_cities_del(self, n):
    self.del_cities["n"] = n

  def set_cities_to_del(self, to_del):
    self.del_cities["to_del"] = to_del

  def set_cities_add(self, n):
    self.add_cities["n"] = n

  def set_cities_move(self, n):
    self.move_cities["n"] = n

  def set_cities_to_move(self, to_move):
    self.move_cities["to_move"] = to_move

  def set_old_dist(self, old_dist):
    self.old_dist = old_dist

  def set_has_changed(self):
      self.changed = True

  def aux_changed(self):
      self.changed = False



In [ ]:
import numpy as np
import random

#Función: Agrega ciudades de manera random

#Input:
#   x: Array actual de coordenadas x de las ciudades
#   y: Array actual de coordenadas y de las ciudades
#   n_Add: Cantidad de ciudades a agregar

#Output:
#   new_x: Nuevo array de coordenadas x de las ciudades
#   new_y: Nuevo array de coordenadas y de las ciudades

def add_cities(x, y, n_add):

  #Crear n_Add nuevos pares de coordenadas para las nuevas ciudades.
  #Valor máximo para las coordenadas: 350
  add_x = np.array([random.randint(0, 250) for _ in range(n_add)])
  add_y = np.array([random.randint(0, 250) for _ in range(n_add)])

  #Concatenar coordenadas antiguas con las nuevas
  new_x = np.concatenate((x, add_x))
  new_y = np.concatenate((y, add_y))

  return new_x, new_y

In [ ]:
import numpy as np

#Función: Eliminar ciudades

#Input:
#   x: Array actual de coordenadas x de las ciudades
#   y: Array actual de coordenadas y de las ciudades
#   to_del: Índices de las ciudades a eliminar

#Output:
#   x: Nuevo array de coordenadas x de las ciudades, con las eliminaciones realizadas
#   y: Nuevo array de coordenadas y de las ciudades, con las eliminaciones realizadas

def del_cities(x, y, to_del):

  x = np.delete(x, to_del)
  y = np.delete(y, to_del)

  return x, y

In [ ]:
import numpy as np
import random

#Función: Modificar las coordenadas (mover) de las ciudades dadas

#Input:
#   x: Array actual de coordenadas x de las ciudades
#   y: Array actual de coordenadas y de las ciudades
#   to_move: Índices de las ciudades a mover

#Output:
#   x: Nuevo array de coordenadas x de las ciudades, con los movimientos realizados
#   y: Nuevo array de coordenadas y de las ciudades, con los movimientos realizados

def move_cities(x, y, to_move):

  #Cantidad de ciudades a mover
  n_move = len(to_move)

  #Crear n_move nuevos pares de coordenadas para las ciudades a mover
  #Valor máximo para las coordenadas: 250
  move_x = np.array([random.randint(0, 250) for _ in range(n_move)])
  move_y = np.array([random.randint(0, 250) for _ in range(n_move)])
  x[to_move] = move_x
  y[to_move] = move_y

  return x, y

In [ ]:
import numpy as np

#Función: Establece las feromonas de las ciudades agregadas con valor tau0

#Input:
#   tau: Matriz de feromonas
#   n_add: Cantidad de ciudades agregadas
#   tau0: Valor inicial de las feromonas

#Output: Nueva matriz de feromonas

def change_add_pheromones(tau, n_add, tau0):

  new_tau = np.full((tau.shape[0] + n_add, tau.shape[1] + n_add), tau0) #Crear matriz con el nuevo tamaño con todos valores tau0

  # Copiar la matriz tau original al espacio correspondiente en la nueva matriz tau
  new_tau[:tau.shape[0], :tau.shape[1]] = tau

  return new_tau

In [ ]:
import numpy as np

#Función: Elimina las feromonas de la matriz de feromonas de las ciudades eliminadas

#Input:
#   tau: Matriz de feromonas
#   to_del: Índices de ciudades eliminadas

#Output: Nueva matriz de feromonas

def change_del_pheromones(tau, to_del):

    # Crea un array de índices que no deben ser eliminados
    keeping_indices = np.setdiff1d(np.arange(tau.shape[0]), to_del)

    # Selecciona las filas y columnas que no deben ser eliminadas
    tau = tau[keeping_indices][:, keeping_indices]

    return tau


In [ ]:
import numpy as np

#Función: Modificar feromonas afectadas por movimiento de ciudades

#Input:
#   tau: Matriz de feromonas
#   changed_city: Ciudad que se movió
#   old_dist: Matriz con las distancias previas al movimiento
#   new_dist: Matriz con las distancias posteriores al movimient
#   tau0: Valor inicial de feromonas

#Output: Nueva matriz de feromonas

def change_move_pheromones(tau, changed_city, old_dist, new_dist, tau0):

    #Calcula el % de cambio de las distancias. Si nueva_distancia > antigua_distancia, % < 0
    rate = (old_dist[changed_city, :] / new_dist[changed_city, :]) - 1

    #La distancia entre una ciudad y ella misma será 0, por lo que existirán divisiones por 0 que darán nan como resultado
    #Aplicar valor 0 a estos casos
    rate[np.isnan(rate)] = 0

    #Modificar el valor de las feromonas, aumentando o disminuyendo según % de cambio
    tau[changed_city, :] = tau[changed_city, :] * (1 + rate)
    tau[:, changed_city] = tau[:, changed_city] * (1 + rate)

    #Aplicar valor tau0 a los valores de feromonas negativos
    tau[tau < 0] = tau0

    return tau

In [ ]:
import numpy as np

# Función: Actualiza la matriz de feromonas según las ciudades eliminadas/agregadas/movidas

#Input:
#   model: Objeto Model, con información sobre el estado actual
#   tau: Matriz de feromonas
#   tau0: Valor inicial de las feromonas

#Output: Matriz de feromonas actualizada

def change_pheromones(model, tau, tau0):

  #Actualiza las feromonas según las ciudades eliminadas
  tau = change_del_pheromones(tau, model.get_cities_to_del())
  #Actualiza las feromonas según las ciudades agregadas
  tau = change_add_pheromones(tau, model.get_cities_add(), tau0)

  #Por cada ciudad movida, actualiza la matriz de feromonas
  for i in range(model.get_cities_move()):
    city = model.get_cities_to_move()[i]
    tau = change_move_pheromones(tau, city, model.get_old_dist(), model.get_dist(), tau0)

  return tau

In [ ]:
import numpy as np
import random

#Función: Crea los parámetros (Movimientos, eliminaciones y adiciones) para un nuevo modelo, siguiente al pasado por parámetro

#Input:
#   model: Modelo base para los nuevos parámetros

#Output:
#   x: Nuevo array de coordenadas x
#   y: Nuevo array de coordenadas y
#   delete: Diccionario con la información sobre las eliminaciones
#           n: Cantidad de ciudades a eliminar
#           to_del: Array con las ciudades a eliminar
#   add: Diccionario con la información sobre las adiciones
#           n: Cantidad de ciudades a agregar
#   move: Diccionario con la información sobre los movimientos
#           n: Cantidad de ciudades a mover
#           to_move: Array con las ciudades movidas

def update_dynamic_aco(model):

  x = model.get_x()
  y = model.get_y()
  delete = {"n": 0,
            "to_del": np.array([])}
  add = {"n": 0}
  move = {"n": 0,
          "to_move": np.array([])}

  n_cities = model.get_n()


  max_cities_del = 3 #Cantidad máxima de ciudades a eliminar
  min_cities = 3 #Cantidad mínima de ciudades existentes

  n_del = random.randint(0, max_cities_del) #Cantidad de ciudades a eliminar

  if ((n_cities - 1) - n_del) <= 0: #Controlamos que n_del no sea mayor que n_cities (sin contar el deposito), de ser asi sobrescribimos
    n_del = 1 if n_cities > min_cities else 0
  to_del = np.array(random.sample(range(1, n_cities - 1), n_del)) #Ciudades a eliminar



  if n_cities - n_del >= min_cities and n_del >= 1:
    x, y = del_cities(x, y, to_del)

    delete["n"] = n_del
    delete["to_del"] = to_del

    n_cities = len(x)

  max_cities_add = 3 #Cantidad máxima de ciudades a agregar
  n_add = random.randint(0, max_cities_add) #Cantidad de ciudades a agregar
  x, y = add_cities(x, y, n_add)
  add["n"] = n_add
  n_cities = len(x)

  max_cities_move = n_cities - 1 #Cantidad máxima de ciudades a mover (no puede ser el deposito)
  n_move = random.randint(1, max_cities_move - 1) #Cantidad de ciudades a mover, como mínimo 1
  to_move = np.array(random.sample(range(1, n_cities - 1), n_move)) #Ciudades a mover
  move["n"] = n_move
  move["to_move"] = to_move
  old_dist = city_distances(x, y) #Distancia antes de los movimientos

  x, y = move_cities(x, y, to_move)


  return x, y, delete, move, add, old_dist



In [ ]:
import numpy as np
import random

#Función: Crea todos los modelos a utilizar por las estrategias

#Input:
#   x_0: Coordenadas iniciales en x
#   y_0: Coordenadas iniciales en y
#   n_model: Cantidad de modelos a crear
#   change_prob: Probabilidad de que exista un cambio respecto al modelo anterior, debe estar entre 0 y 1

#Output: Array con los modelos en orden de uso

def create_models(x_0, y_0, n_model, change_prob):

  model = Model(x_0, y_0) #Crear modelo inicial
  #model.set_has_changed()
  models = np.array([model])

  for i in range(1, n_model):
    if random.random() <= change_prob:
      x, y, delete, move, add, old_dist = update_dynamic_aco(models[i - 1]) #Crear parámetros para el nuevo modelo
      model = Model(x, y)
      model.set_cities_del(delete["n"])
      model.set_cities_to_del(delete["to_del"])
      model.set_cities_move(move["n"])
      model.set_cities_to_move(move["to_move"])
      model.set_cities_add(add["n"])
      model.set_old_dist(old_dist)
      model.set_has_changed() #Indicar que el modelo cambió respecto al anterior
    else:
      model = Model(models[i - 1].get_x(), models[i - 1].get_y()) #Si no se realiza ninguna modificación, el nuevo modelo es idéntico al anterior

    models = np.append(models, model)

  return models

In [ ]:
import numpy as np
import random
import numpy as np
import itertools
from tqdm import tqdm

#Genera datos aleatorios para los modelos

def generate_data(data_size,seq_len, seed):
        """
        :return: Set of points_list ans their One-Hot vector solutions
        """
        points_list = []
        np.random.seed(seed)
        data_iter = tqdm(range(data_size), unit='data')
        for i, _ in enumerate(data_iter):
            data_iter.set_description('Data points %i/%i' % (i+1, data_size))
            points_list.append(np.random.random((seq_len, 2)))
        solutions_iter = tqdm(points_list, unit='solve')


        return points_list

#Función: Crea todos los modelos a utilizar por las estrategias utilizando datos aleatorios

#Input:
#   x_0: Coordenadas iniciales en x
#   y_0: Coordenadas iniciales en y
#   n_model: Cantidad de modelos posibles a crear
#   change_prob: Probabilidad de que exista un cambio respecto al modelo anterior, debe estar entre 0 y 1
#   time_limit: Cantidad de tiempo maximo disponible para ejecutar
#Output: Array con los modelos en orden de uso y Array los tiempos de activacion de los mismos

def create_models_timed(data_size, seq_len, seed, n_model, change_prob, time_limit):
  data = generate_data(data_size,seq_len, seed) #Generamos los datos normalizados
  model_output = []
  for points in data:
    point_list = points * 100 # Escalamos los datos en el rango 0 - 100
    l = point_list.flatten('F')
    x_0,y_0 = np.array_split(l,2)

    model = Model(x_0, y_0) #Crear modelo inicial
    model.set_has_changed()
    models = np.array([model])

    models_changed = np.array([model])

    for i in range(1, n_model):
      if random.random() <= change_prob:
        x, y, delete, move, add, old_dist = update_dynamic_aco(models[i - 1]) #Crear parámetros para el nuevo modelo
        model = Model(x, y)
        model.set_cities_del(delete["n"])
        model.set_cities_to_del(delete["to_del"])
        model.set_cities_move(move["n"])
        model.set_cities_to_move(move["to_move"])
        model.set_cities_add(add["n"])
        model.set_old_dist(old_dist)
        model.set_has_changed() #Indicar que el modelo cambió respecto al anterior
        models_changed = np.append(models_changed, model)
      else:
        model = Model(models[i - 1].get_x(), models[i - 1].get_y()) #Si no se realiza ninguna modificación, el nuevo modelo es idéntico al anterior

      models = np.append(models, model)

    time_window = time_limit / (len(models_changed)-1)
    times = []
    for t in range(1, len(models_changed)):
      time_gap = time_window + (round(random.uniform(0.1*time_window, 0.5 * time_window), 2) - 0.4 * time_window)
      times.append(time_gap)
    times = np.cumsum(times)

    model_output.append([models_changed, times, time_limit])
  return model_output

In [ ]:
from scipy import integrate

#Función: Obtener el "área bajo la curva" del conjunto discreto de costos, mediante el método de los trapecios

#Input:
#   costs: Array de los costos a lo largo de las iteraciones

#Output: Área bajo la curva

def get_area(costs):
  return integrate.trapz(costs)

In [ ]:
class Period:
    def __init__(self, it, best_cost, worst_cost, it_best_cost):
        self.it = it
        self.best_cost = best_cost
        self.worst_cost = worst_cost
        self.it_best_cost = it_best_cost
        self.worst_cost_by_heuristic = None
        self.accuracy = None
        self.stability = None
        self.reactivity = None
        self.fit_adapt_speed = None

    # Getters
    def get_it(self):
        return self.it

    def get_best_cost(self):
        return self.best_cost

    def get_worst_cost(self):
        return self.worst_cost

    def get_it_best_cost(self):
        return self.it_best_cost

    def get_worst_cost_by_heuristic(self):
        return self.worst_cost_by_heuristic

    def get_accuracy(self):
        return None
        #return round(self.accuracy * 100) / 100

    def get_stability(self):
      if self.stability is None:
        return None
      return round(self.stability * 100) / 100

    def get_reactivity(self):
        return None
        #return self.reactivity

    def get_fit_adapt_speed(self):
      if self.fit_adapt_speed is None:
        return None
      return round(self.fit_adapt_speed * 100) / 100

    # Setters

    def set_accuracy(self, accuracy):
        self.accuracy = accuracy

    def set_stability(self, stability):
        self.stability = stability

    def set_reactivity(self, reactivity):
        self.reactivity = reactivity

    def set_fit_adapt_speed(self, fit_adapt_speed):
        self.fit_adapt_speed = fit_adapt_speed

    def set_worst_cost_by_heuristic(self, worst_cost):
      self.worst_cost_by_heuristic = worst_cost

In [ ]:
from tabulate import tabulate

def view_periods(periods):
  # Imprime los periodos en formato de tabla
  print("PERIODOS\n")
  table = [["Iteraciones", "Mejor Costo", "Iteración del Mejor Costo", "Accuracy", "Stability", "Reactivity", "Fit Adapatation Speed"]]
  for period in periods:
      table.append([period.get_it(), str(period.get_best_cost()), str(period.get_it_best_cost()), str(period.get_accuracy()), str(period.get_stability()), period.get_reactivity(), str(period.get_fit_adapt_speed())])

  print(tabulate(table, headers="firstrow", tablefmt="grid", numalign="center"))

In [ ]:
import numpy as np

#Función: Obtener los periodos
#Periodo: Instancias de n o más modelos en donde no hubo modificaciones

#Input:
#   models: Lista con los modelos a analizar
#   costs: Lista con los costos de cada modelo

#Output:
#   periods: Lista con todos los periodos, donde cada uno tiene el número de los modelos y el mejor costo

def get_periods(models, flags, costs):

  n = 1 #Cantidad minima de iteraciones sin cambios para que se considere periodo

  periods = np.array([]) #Array con todos los periodos

  # Información de cada periodo
  it = [] # Iteraciones en las que ocurre
  best_cost = 0 # Mejor costo
  worst_cost = 0 # Peor costo
  worst_cost_by_heuristic = 0 # Peor costo calculado por heuristica
  it_best_cost = 0 # Cantidad de iteraciones necesarias para obtener el mejor costo
  period_model = None # Modelo correspondiente al periodo


  for i in range(len(models)): #Obtener número de iteraciones

    model = models[i]
    flag = flags[i]
    if not flags[i]:
      if i >= 1 and i - 1 not in it: #Agregar el primero del periodo
        it.append(i - 1)

      it.append(i)

      if i == len(models) - 1: #Si se está evaluando el último modelo, agregar el período

        best_cost, worst_cost, it_best_cost = get_best_worst_cost_and_it(it, costs)
        #worst_cost_by_heuristic = get_worst_cost(model)

        periods = np.append(periods, Period(it, best_cost, worst_cost, it_best_cost))


    else:
      if len(it) >= n:
        best_cost, worst_cost, it_best_cost = get_best_worst_cost_and_it(it, costs)

        #worst_cost_by_heuristic = get_worst_cost(model)
        periods = np.append(periods, Period(it, best_cost, worst_cost, it_best_cost))


      it = [i]
      best_cost = 0
      worst_cost = 0
      it_best_cost = 0

  return periods



In [ ]:
import numpy as np
#Función: Obtener el mejor costo de las iteraciones dadas y su índice

#Input:
#   it: Lista de iteraciones
#   costs: Lista con los costos de cada modelo

#Output:
#   best_cost: Mejor costo
#   it_best_cost: Número de iteración de best_cost

def get_best_worst_cost_and_it(it, costs):
    best_cost = min(costs[it]) # Calcular el mejor costo dentro de las iteraciones
    worst_cost = max(costs[it])
    b_cost_it = np.where(costs == best_cost)[0][0] # Encuentra el índice en el arreglo 'costs' donde se encuentra el mínimo costo
    it_best_cost = it.index(b_cost_it) + 1 # Encuentra la posición del índice 'b_cost_it' dentro de la lista 'it', y suma 1 para obtener la iteración
    return best_cost, worst_cost, it_best_cost

In [ ]:
import numpy as np

#Función: Calcular la precisión de una solución. Será un valor en [0, 1]. Mientras más cercano a 1 es, mayor precisión tendrá.

#Input:
#   sol: Mejor solución de la iteración que está siendo evaluada
#   b_sol: Mejor solución encontrada hasta el momento
#   w_sol: Peor solución encontrada hasta el momento

#Output: Costo de accuracy

def accuracy(sol, b_sol, w_sol):

  acc = (sol - w_sol)/ (b_sol - w_sol) #Precisión

  return acc

In [ ]:
#Función: Obtener la precisión (Accuracy) del mejor costo de cada periodo

#Input:
#   periods: Lista con todos los periodos
#   accuracy: Lista con los valores de precisión

#Output: Cada periodo tiene ahora su valor de accuracy

def get_acc_period(periods, accuracy):

  for period in periods:
    for acc in accuracy:
      if period.get_best_cost() == acc["cost"] and acc["it"] - 1 in period.get_it():
        period.set_accuracy(acc['acc'])
        break

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

#Función: Calcular la medida fitness degradation, indica si las soluciones
#         tienden a ser mejores (Pendiente positiva) o peores (Pendiente negativa)

#Inputs:
#   models: Array con los modelos
#   costs: Array con los costos de cada repetición del algoritmo

def fitness_degradation(periods):

  x = np.array([i+1 for i in range(periods.shape[0])])
  y = periods['Mejor accuracy promedio'].tolist()

  res = linregress(x, y)

  meth = periods['Metodo'].values[0]
  model = periods['Modelo'].values[0]
  n_ants = periods['Cantidad de hormigas'].values[0]


  name = f"ACO {meth}\nModelo: {model}\nHormigas: {n_ants}\nPendiente: {round(res.slope*100000)/100000}\nCorrelación: {round((res.rvalue**2)*100)/100}"

  plt.plot(x, y, 'o', label='Accuracy')
  plt.plot(x, res.intercept + res.slope*x, 'r', label='fitted line')
  plt.title(name)
  plt.legend()
  plt.show()
  print()
  return pd.Series({'Fitness Degradation': res.slope,
                    'Correlacion': res.rvalue**2})

In [ ]:
#Función: Calcular la medida stability, indica el impacto de los cambios en el entorno.
#         Valor entre 0 y 1
#         0: Muy estable, 1: Inestable
#   Supone que el dataframe tiene las ejecuciones intercaladas por cada método

#Input:
#   accuracy: DataFrame con los datos de accuracy a los que se le calculará stability.

#Output: Valor de stability

prev_last_accuracy = None #Accuracy del periodo anterior al evaluado
prev_meth = None #Método del periodo anterior al evaluado

def stability(accuracy):
  global prev_last_accuracy, prev_meth
  stability = 0
  meth = accuracy['Metodo'].values[0]

  if not prev_meth:
    prev_meth = meth
  elif meth != prev_meth:
    prev_meth = meth
    prev_last_accuracy = None

  if prev_last_accuracy:
    first_accuracy = accuracy['Accuracy'].values[0]
    stability = prev_last_accuracy - first_accuracy

  prev_last_accuracy = accuracy['Accuracy'].values[-1]

  return stability if stability > 0 else 0

In [ ]:
#Función: Calcular medida rectivity, indica la capacidad del periodo a adaptarse rápido a cambios en el entorno
#         Valor natural entre 1 y la cantidad de periodos
#         Mientras más cercano a 1 el valor, más rapidamente se adaptó a los cambios el periodo evaluado
#         Si react = inf, indica que el periodo evaluado no encontró un periodo siguiente con suficiente precisión según las condiciones del algoritmo (1 - e).

#Input:
#   accuracy: DataFrame con los datos de accuracy a los que se les calculará reactivity.

#Output: Valor de reactivity

def reactivity(accuracy):

  e = 0.02 # Tolerancia a la variación en la precisión al evaluar la reactividad de los periodos

  accuracy = accuracy.sort_values(by=['Iteracion'], ascending=[True])
  first_accuracy = accuracy['Accuracy'].values[0]
  div = pd.DataFrame(first_accuracy / accuracy['Accuracy']).reset_index()['Accuracy'].tolist()
  reactivity = -1 # La primer ocurrencia en las divisiones que sea mayor o igual al umbral será la reactivity
  for i, d in enumerate(div):
    if d >= 1-e and i != 0:
      reactivity = i + 1
      break

  return reactivity

In [ ]:
#Función: Calcular medida fitness adaptation speed, que indica que tan rápido el periodo se adapta a los cambios

#Input:
# periods: Dataframe del periodo al que se le calculará FAS

#Output: Valor de FAS

def fitness_adaptation_speed(period):

  first_cost = period['Mejor costo'].values[0]
  best_cost = period['Mejor costo'].min()
  delta_cost = abs(best_cost - first_cost)

  first_it = period['Iteracion'].values[0]
  idx_b_cost = period['Mejor costo'].idxmin()
  best_cost_it = period.loc[idx_b_cost]['Iteracion']

  mobility = best_cost_it - first_it

  if mobility == 0:
    return 0

  fas = delta_cost / mobility

  return fas


In [ ]:
import numpy as np
import time

#Clase cronometro para llevar registro del tiempo
class TimeHelper:

  def __init__(self, name):
    self.name = name
    self.time = 0

  def reset(self):
    self.time = 0

  def increment(self, time_increment):
    self.time += time_increment

#Clase para manejar accesos y tiempos de la funcion
class ObjFunction:

  def __init__(self, cost_function, limit = None):
    self.cost_function = cost_function
    self.limit = limit
    self.interactions = 0
    self.timeHelperFunction = TimeHelper("Function")
    #self.timeHelperLocal = TimeHelper("Local")

  #Aunque se llame "over", incluimos la igualdad como condicion de corte.
  def over_limit(self):
    return self.limit is not None and self.limit <= self.interactions

  def use_function(self, tour, model, scope):
    self.interactions+=1
    time_start = time.time()
    cost = self.cost_function(tour, model)
    total_time = time.time() - time_start
    if scope == "F": #F para Function
      self.timeHelperFunction.increment(total_time)
    #elif scope == "L": # L para Local
    #  self.timeHelperLocal.increment(total_time)
    return cost

  def reset(self):
    self.interactions = 0
    self.timeHelperFunction.reset()
    self.timeHelperLocal.reset()

Una "hormiga" es una entidad abstracta que representa un agente que busca soluciones en un espacio de búsqueda. Cada hormiga construye una solución al problema es decir, una hormiga construye un recorrido que visita todas las ciudades exactamente una vez y regresa al punto de partida.

Las hormigas siguen una regla de selección para determinar qué ciudad visitar a continuación. Esta regla depende de la influencia de las feromonas y de la información heurística.

Después de construir una solución, la hormiga deposita feromonas en los bordes del grafo que ha recorrido. Las feromonas depositadas influyen en las decisiones futuras de otras hormigas.

A través de múltiples iteraciones, las hormigas colaboran para explorar el espacio de soluciones y gradualmente convergen hacia una solución óptima o cercana a la óptima.

n_ant representa el número de hormigas que participan en la búsqueda de soluciones en cada iteración del algoritmo. Más hormigas pueden explorar un espacio de búsqueda más amplio, pero también pueden aumentar el tiempo de ejecución del algoritmo. Un número adecuado de hormigas es crucial para un equilibrio óptimo entre exploración y explotación.

Por "feromona" se habla de una sustancia ficticia utilizada para modelar el comportamiento de las hormigas y comunicar información sobre la calidad de las soluciones en un espacio de búsqueda. Una especie de "memoria colectiva" del comportamiento de las hormigas. Indican la calidad de los caminos o soluciones en el espacio de búsqueda del problema que se está resolviendo. Cuanto más fuerte es la feromona en un camino, más probable es que las hormigas futuras lo elijan.

Las feromonas se actualizan dinámicamente durante el proceso de búsqueda. Cuando una hormiga completa su recorrido y encuentra una solución, deposita feromonas en los bordes del grafo que ha recorrido. La cantidad de feromona depositada suele ser inversamente proporcional a la calidad de la solución: mejores soluciones reciben más feromona.

A medida que las hormigas siguen explorando, las feromonas se evaporan gradualmente, lo que simula el paso del tiempo y evita la convergencia prematura hacia soluciones subóptimas.

Q es un parámetro que controla la cantidad de feromonas que se deposita en el camino seguido por una hormiga después de completar un recorrido. Q influye en la tasa a la que las feromonas se acumulan en los bordes del grafo. Valores más altos de Q significan que se deposita más feromona,resultando en una mayor influencia de las feromonas en la elección de las rutas por parte de las hormigas. Por otro lado, valores más bajos de Q disminuirán el impacto de las feromonas en la elección de rutas. La selección adecuada de Q es importante para el desempeño óptimo del algoritmo ACO y puede depender del problema específico que se esté abordando.

tau0 es el nivel inicial de feromonas en todos los bordes del grafo al comienzo del algoritmo. tau0 establece una base inicial para las feromonas. Un valor adecuado de tau0 puede ayudar a equilibrar la exploración y la explotación en las primeras etapas del algoritmo.

rho representa la tasa a la que las feromonas se evaporan entre iteraciones del algoritmo. La evaporación de feromonas simula el paso del tiempo y ayuda a evitar la convergencia prematura hacia soluciones subóptimas. Un valor adecuado de rho controla la rapidez con la que las feromonas disminuyen.

En ACO, hay dos influencias principales que guían el comportamiento de las hormigas al elegir su próximo movimiento:

  Influencia de las Feromonas (**alpha**): Este parámetro controla cuánto peso se le da a la información de feromonas al tomar decisiones. Un valor alto de alpha hace que las hormigas confíen más en las feromonas al elegir su siguiente ciudad.

  Influencia de la Información Heurística (**beta**): Este parámetro controla cuánto peso se le da a la información heurística, que en el caso del TSP suele ser la distancia entre ciudades. Un valor alto de beta hace que las hormigas confíen más en la información de distancia al elegir su siguiente ciudad.

In [ ]:
import numpy as np
import time
import copy


#Funcion:implementa una versión básica de Ant Colony Optimization para resolver TSP.

#Input:
#      models: Array con los modelos (distribuciones de ciudades) predefinidos que tomará el algoritmo a lo largo de las iteraciones
#      cost_function: Una función que calcula el costo de un recorrido dado. En este caso, cost_function se refiere a tour_length, que calcula la longitud total del recorrido en el TSP.
#      max_it: Número máximo de iteraciones que el algoritmo realizará.
#      n_ant: Número de hormigas, el tamaño de la población.
#      Q: controla la cantidad de feromonas que se depositan en las aristas durante la actualización de feromonas.
#      tau0: Nivel inicial de feromonas.
#      alpha: controla la influencia de las feromonas en la decisión de las hormigas.
#      beta:  controla la influencia de la información heurística (distancias entre ciudades) en la decisión de las hormigas.
#      rho: Tasa de evaporación de feromonas.
#      L: Capacidad de cada hormiga.
#      time_limit: El tiempo maximo posible de ejecucion.



def simple_aco(timed_models, cost_function, max_it, n_ant, Q, tau0, alpha, beta, rho, L):
    figs = plt.get_fignums()  # Obtiene los números de las figuras ya existentes
    num_figs = len(figs)  # Cuenta la cantidad de figuras

    acc = np.array([]) # Arreglo con las precisiones de cada iteración

    print('\nSIMPLE ACO\n')

    models, times, time_limit = timed_models
    model_index = 0
    time_index = 0
    model_change = False
    last_model = False #Flag para dejar de controlar tiempos

    model = models[model_index] # Modelo de inicio
    models_by_iteration = np.array([])
    models_by_iteration_flags = np.array([])
    n_var = model.get_n()  # Número de ciudadesvariables del modelo
    eta = 1 / model.get_dist()  # Calcula la matriz de información heurística, que es el inverso de las distancias.  (Tengo que arreglar un warning de division por cero)
    tau = tau0 * np.ones((n_var, n_var))  #  Inicializa la matriz de feromonas con valores iguales a tau0.

    best_cost = np.array([float('inf') for _ in range(max_it)])  # Arreglo para almacenar los mejores costos en cada iteración
    worst_cost = np.array([float('-inf') for _ in range(max_it)])  # Arreglo para almacenar los peores costos en cada iteración
    best_sols= []


    limit_flag = False # Flag de cantidad de iteraciones
    time_flag = False # Flag de tiempo
    access_flag = False # Flag de accesos

#    Se define una hormiga vacía como un diccionario con dos claves: 'Tour' y 'Cost'.
#    'Tour' será una lista que representará el recorrido de ciudades que realizará la hormiga.
#    'Cost' representará el costo total del recorrido de la hormiga.
    empty_ant = {'Tour': [], 'Cost': []}

#   Se crea una lista llamada ants que contendrá n_ant hormigas.
#   Utiliza una comprensión de lista ([...]) para llenar la lista con copias de la hormiga vacía (empty_ant). Cada hormiga en la lista es independiente y tiene su propio recorrido y costo.
    ants = [empty_ant.copy() for _ in range(n_ant)]

    best_sol = {'Cost': float('inf')}  #  Inicializa la mejor solución encontrada con un costo infinitamente grande, dependiente del periodo.
    b_cost = float('inf') # Mejor costo, independiente del periodo
    w_cost = float('-inf') # Peor costo, independiente del periodo


#   Antes de comenzar, controlamos que se pueda visitar y volver con todos los puntos desde el deposito
    for v in range(1,n_var):
      single_tour = [0,v]
      costo = cost_function.use_function(single_tour, model, "F")
      if L < costo:
        raise RuntimeError(f"La ciudad {v} se encuentra demasiado lejos: L = {L}, Costo de {single_tour} = {costo} ")

    it = -1
    start_time = time.time()

    # Bucle principal de ACO
    while not limit_flag and not time_flag and not access_flag:  #Se ejecutara el algoritmo de ACO durante un número máximo de iteraciones.

        it += 1
         #Cambiar el modelo por el siguiente, si este presentó cambios
        model = models[model_index]
        print(model.has_changed())
        if model_change:
          model = models[model_index]
          n_var = model.get_n()
          eta = 1 / model.get_dist()
          tau = change_pheromones(model, tau, tau0)
          best_sols.append(best_sol)
          best_sol = {'Cost': float('inf')}
          model.set_has_changed()
          model_change = False
          models_by_iteration = np.append(models_by_iteration, model) #Agrega el modelo a la lista de modelos por iteracion
          models_by_iteration_flags = np.append(models_by_iteration_flags, True)
        #Como hubo un cambio de modelo, controlamos que se pueda visitar y volver todos los puntos desde el deposito
          for v in range(1,n_var):
            single_tour = [0,v]
            costo = cost_function.use_function(single_tour, model, "F")
            if L < costo:
              raise RuntimeError(f"La ciudad {v} se encuentra demasiado lejos: L = {L}, Costo de {single_tour} = {costo} ")

        else:
          model.aux_changed()
          print(model.has_changed())
          models_by_iteration = np.append(models_by_iteration, model) #Agrega el modelo a la lista de modelos por iteracion
          models_by_iteration_flags = np.append(models_by_iteration_flags, False)

        # Mover las hormigas
        for k in range(n_ant): #Este bucle itera sobre todas las hormigas en la población.
            depo = [0] # El punto considerado "deposito" para el problema, tomamos el punto "0" como convención.
            ants[k]['Tour'] = depo  # Ciudad inicial de cada hormiga k. Se elige una ciudad aleatoria como punto de partida.
            pending = n_var-1 # Ciudades que faltan por recorrer ignorando el deposito.
            while (pending != 0): #permite a la hormiga moverse a través de las ciudades restantes.
                i = ants[k]['Tour'][-1]  # Última ciudad visitada por la hormiga
                P = (tau[i, :] ** alpha) * (eta[i, :] ** beta)  # Cálculo de las probabilidades

                #Construimos el filtro para determinar que nodos no pueden ser visitados dependiendo del ultimo lugar visitado por la hormiga.
                if i==0:
                  filter = ants[k]['Tour']  # Si la hormiga "esta" en el deposito, filtramos todos los lugares ya visitados (incluyendo el deposito)
                else:
                  filter =  [i for i in ants[k]['Tour'] if i != 0] # Si la hormiga "no esta" en el deposito, se filtran todos los lugares exceptuando el deposito.
                #Agregamos al filtro las ciudades que involucren sobrepasar el limite de longitud
                sub = sub_touring(ants[k]['Tour'], 0)
                #print("Tour separado: ",sub)
                #print("Costo ultimo sub: ",cost_function(sub[-1]))
                for c in range(1,n_var-1):
                  s = copy.deepcopy(sub[-1])
                  if c not in filter: # Para todas las ciudades que aun no estan filtradas verificamos que puedan ir y llegar al deposito sin superar la capacidad.
                    s.append(c)
                    #s.append(0)
                    cost_s = cost_function.use_function(s,model,"F")
                    #print("Subtour candidato: ",s)
                    #print("Costo asociado: ", cost_s)
                    if L < cost_s: #Si se supera la capacidad, se filtra esta ciudad.
                        filter.append(c)
                #Aplicamos el filtro
                P[filter] = 0
                P = P / np.sum(P)  # Normalización de las probabilidades
                j = roulette_wheel_selection(P)  # Próxima ciudad a visitar
                ants[k]['Tour'].append(j)  # Agregar la ciudad al tour de la hormiga
                if j != 0: #Si la ciudad visitada no es el deposito
                  pending = pending - 1 #Decrementamos las ciudades restantes


            ants[k]['Cost'] = cost_function.use_function(ants[k]['Tour'], model, "F")  # Calcular el costo del tour

            if best_cost[it] > ants[k]['Cost']:
              best_cost[it] = ants[k]['Cost']  # Almacenar el mejor costo de esta iteración
            elif worst_cost[it] < ants[k]['Cost']:
              worst_cost[it] = ants[k]['Cost'] # Almacenar el peor costo de esta iteración

            if ants[k]['Cost'] < best_sol['Cost']:  # Actualizar la mejor solución si es necesario. Este costo pertenece al periodo actual
              best_sol = ants[k].copy()
              it_best = it + 1

            if ants[k]['Cost'] < b_cost: # Actualizar la mejor o peor solución general
              b_cost = ants[k]['Cost']
            elif ants[k]['Cost'] > w_cost:
              w_cost = ants[k]['Cost']

        # Actualizar las feromonas
        for k in range(n_ant):  #Se recorre cada hormiga (k) en el bucle principal.
            tour = ants[k]['Tour']  #Se obtiene el recorrido de la hormiga k.
            #tour.append(tour[0])  # Se agrega la primera ciudad al final del recorrido para formar un circuito cerrado.
            for l in range(n_var):  #Se itera sobre las ciudades
                #Se obtienen los índices de las ciudades i y j en la matriz de feromonas, restando 1 para ajustar por el hecho de que los índices en Python comienzan en 0.
                i = tour[l] - 1
                j = tour[l + 1] - 1
                tau[i, j] += Q / ants[k]['Cost']  #Se actualiza la cantidad de feromonas entre las ciudades i y j al agregar una cantidad proporcional a Q dividido por el costo del recorrido de la hormiga k

        tau = (1 - rho) * tau  # Evaporación de las feromonas

        #best_cost[it] = best_sol['Cost']  # Almacenar el mejor costo de esta iteración

        print(f'Iteración {it+1}: Mejor Costo = {best_cost[it]}')  # Muestra información de la iteración

        plt.figure(num_figs + 1)
        graph_name = f'Solución ACO Simple\nIteración {it+1}: Mejor Costo = {best_cost[it]}'
        plot_solution(best_sol['Tour'], model, graph_name)  # Graficar la solución
        graph_phe= f'Feromonas ACO Simple\nIteración {it+1}'
        plot_pheromones(model,tau,graph_phe)

        #acc_i = accuracy(best_cost[it], b_cost,  w_cost, it+1) # Calcula accuracy para la iteración actual
        #acc = np.append(acc, acc_i) # Agrega la accuracy de la iteración actual a la lista

        limit_flag = it == max_it - 1
        time_flag = time.time() - start_time  >= time_limit
        access_flag = cost_function.over_limit()

        current_time = time.time() - start_time

        if not last_model:
          if current_time > times[time_index]:
            model_index+= 1
            time_index+=1
            model_change = True
            if time_index == len(times)-1:
                last_model=True

        plt.pause(0.01)  # Pausa para mostrar la gráfica



    plt.figure()
    plt.plot(best_cost[:it+1], linewidth=2)
    plt.xlabel('Iteración')
    plt.ylabel('Mejor Costo')
    plt.title('ACO Simple')
    plt.grid(True)

    plot_pheromones(model, tau, 'Feromonas ACO Simple')  # Graficar las feromonas

    flag = ''
    if limit_flag:
      flag = 'limit_flag'
    elif time_flag:
      flag = 'time_flag'
    else:
      flag = 'access_flag'

    best_sols.append(best_sol)
    solution = [best_cost[-1], it_best, best_cost[:it+1], worst_cost[:it+1], models_by_iteration, models_by_iteration_flags, flag, best_sols]  # Mejor costo, número de iteración correspondient, mejores costos de cada iteración y modelos utilizados en cada iteración
    plt.pause(1)  # Pausa antes de cerrar la gráfica
    return solution


In [ ]:
#Funcion: Ordena una lista de hormigas según sus costos.
#Input:
#     ants: Una lista de hormigas, donde cada hormiga está representada por un diccionario con un 'Tour' y 'Cost'.
#Output: Retorna una lista ordenada de hormigas.

def sort_ants(ants):
    # Caso base: si hay 1 o menos hormigas, la lista ya está ordenada
    if len(ants) <= 1:
        return ants
    else:
        # Se selecciona la primera hormiga como pivote
        piv = ants[0]['Cost']
        # Se crean dos listas vacías para las hormigas menores y mayores que el pivote
        l = []
        g = []
        # Se recorren las hormigas a partir de la segunda
        for i in range(1, len(ants)):
            # Si el costo de la hormiga es menor o igual que el pivote, se agrega a la lista de menores
            if ants[i]['Cost'] <= piv:
                l.append(ants[i])
            else:
                # Si es mayor, se agrega a la lista de mayores
                g.append(ants[i])
        # Se aplica recursivamente la función a las listas de menores y mayores
        l = sort_ants(l)
        g = sort_ants(g)
        # Se concatenan las listas de menores, el pivote y las mayores en ese orden
        sorted_ants = l + [ants[0]] + g
        return sorted_ants


In [ ]:
#Funcion: Selecciona las mejores n hormigas con los costos más bajos.
#Input:
#    ants: Una lista de hormigas, donde cada hormiga está representada por un diccionario con un 'Tour' y 'Cost'.
#    n: Número de hormigas élite a seleccionar.
#Output:Retorna una lista de hormigas élite.

def elitist_ants(ants, n):
    # Inicializa una lista vacía para almacenar las hormigas elitistas
    elit_ants = []

    # Inicializa el valor máximo de costo como -1 (un valor menor que cualquier posible costo)
    max_cost = -1

    # Itera sobre las hormigas y selecciona las mejores
    i = 0
    while len(elit_ants) < n and i < len(ants):
        ant = ants[i]  # Obtiene la hormiga actual
        if round(ant['Cost'] * 1000) / 1000 > round(max_cost * 1000) / 1000:
            # Si el costo de la hormiga actual es mayor que el costo máximo encontrado hasta ahora
            # Se añade a la lista de hormigas elitistas y se actualiza el costo máximo
            elit_ants.append(ant)
            max_cost = ant['Cost']
        i += 1  # Incrementa el índice para pasar a la siguiente hormiga

    return elit_ants  # Devuelve la lista de hormigas elitistas


In [ ]:
#Funcion:implementa una variante de ACO, "ACO Elitista". Esta variante prioriza la explotación de las mejores soluciones encontradas hasta el momento para resolver TSP.

#Input:
#      models: Array con los modelos (distribuciones de ciudades) predefinidos que tomará el algoritmo a lo largo de las iteraciones
#      cost_function: Una función que calcula el costo de un recorrido dado. En este caso, cost_function se refiere a tour_length, que calcula la longitud total del recorrido en el TSP.
#      max_it: Número máximo de iteraciones que el algoritmo realizará.
#      n_ant: Número de hormigas, el tamaño de la población.
#      Q: controla la cantidad de feromonas que se depositan en las aristas durante la actualización de feromonas.
#      tau0: Nivel inicial de feromonas.
#      alpha: controla la influencia de las feromonas en la decisión de las hormigas.
#      beta:  controla la influencia de la información heurística (distancias entre ciudades) en la decisión de las hormigas.
#      rho: Tasa de evaporación de feromonas.
#      r: Cantidad de hormigas consideradas élite.
#      L: Capacidad de cada hormiga.
#      time_limit: El tiempo maximo posible de ejecucion.

import time
import copy

def elitist_aco(timed_models, cost_function, max_it, n_ant, Q, tau0, alpha, beta, rho, r, L):
    figs = plt.get_fignums()
    num_figs = len(figs)

    acc = np.array([]) # Arreglo con las precisiones de cada iteración

    if r == 1:
      print('\nELITIST ACO\n')
    else:
      print('\nRANK ACO\n')

    models, times, time_limit = timed_models
    model_index = 0
    time_index = 0
    model_change = False
    last_model = False #Flag para dejar de controlar tiempos

    n_best_ants = int(r)  # Número de hormigas consideradas las mejores

    model = models[0] # Modelo de inicio
    models_by_iteration = np.array([])
    models_by_iteration_flags = np.array([])
    n_var = model.get_n()  # Número de ciudadesvariables del modelo
    eta = 1 / model.get_dist()  # Calcula la matriz de información heurística, que es el inverso de las distancias.  (Tengo que arreglar un warning de division por cero)
    tau = tau0 * np.ones((n_var, n_var))  #  Inicializa la matriz de feromonas con valores iguales a tau0.

    best_cost = np.array([float('inf') for _ in range(max_it)])# Arreglo para almacenar los mejores costos en cada iteración
    worst_cost = np.array([float('-inf') for _ in range(max_it)])# Arreglo para almacenar los mejores costos en cada iteración
    best_sols = []


    empty_ant = {'Tour': [], 'Cost': []}  # Estructura de una hormiga vacía
    ants = [empty_ant.copy() for _ in range(n_ant)]  # Lista de hormigas

    best_sol = {'Cost': float('inf')}  # Mejor solución encontrada
    b_cost = float('inf') # Mejor costo, independiente del periodo
    w_cost = float('-inf') # Peor costo, independiente del periodo

    limit_flag = False # Flag de cantidad de iteraciones
    time_flag = False # Flag de tiempo
    access_flag = False # Flag de accesos


#   Antes de comenzar, controlamos que se pueda visitar y volver todos los puntos desde el deposito
    for v in range(1,n_var):
      single_tour = [0,v]
      costo = cost_function.use_function(single_tour, model, "F")
      if L < costo:
        raise RuntimeError(f"La ciudad {v} se encuentra demasiado lejos: L = {L}, Costo de {single_tour} = {costo} ")

    it = -1
    start_time = time.time()

    # Bucle principal de ACO
    while not limit_flag and not time_flag and not access_flag:  #Se ejecutara el algoritmo de ACO durante un número máximo de iteraciones.

        it += 1
        if model_change:
          model = models[model_index]
          n_var = model.get_n()
          eta = 1 / model.get_dist()
          tau = change_pheromones(model, tau, tau0)
          best_sols.append(best_sol)
          best_sol = {'Cost': float('inf')}
          model_change = False
          model.set_has_changed()
          models_by_iteration = np.append(models_by_iteration, model) #Agrega el modelo a la lista de modelos por iteracion
          models_by_iteration_flags = np.append(models_by_iteration_flags, True)

        #Como hubo un cambio de modelo, controlamos que se pueda visitar y volver todos los puntos desde el deposito
          for v in range(1,n_var):
            single_tour = [0,v]
            costo = cost_function.use_function(single_tour, model, "F")
            if L < costo:
              raise RuntimeError(f"La ciudad {v} se encuentra demasiado lejos: L = {L}, Costo de {single_tour} = {costo} ")

        else:
          model.aux_changed()
          models_by_iteration = np.append(models_by_iteration, model) #Agrega el modelo a la lista de modelos por iteracion
          models_by_iteration_flags = np.append(models_by_iteration_flags, False)


        # Mover las hormigas
        for k in range(n_ant): #Este bucle itera sobre todas las hormigas en la población.
            depo = [0] # El punto considerado "deposito" para el problema, tomamos el punto "0" como convención.
            ants[k]['Tour'] = depo  # Ciudad inicial de cada hormiga k. Se elige una ciudad aleatoria como punto de partida.
            pending = n_var-1 # Ciudades que faltan por recorrer ignorando el deposito.
            while (pending != 0): #permite a la hormiga moverse a través de las ciudades restantes.
                i = ants[k]['Tour'][-1]  # Última ciudad visitada por la hormiga
                P = (tau[i, :] ** alpha) * (eta[i, :] ** beta)  # Cálculo de las probabilidades

                #Construimos el filtro para determinar que nodos no pueden ser visitados dependiendo del ultimo lugar visitado por la hormiga.
                if i==0:
                  filter = ants[k]['Tour']  # Si la hormiga "esta" en el deposito, filtramos todos los lugares ya visitados (incluyendo el deposito)
                else:
                  filter =  [i for i in ants[k]['Tour'] if i != 0] # Si la hormiga "no esta" en el deposito, se filtran todos los lugares exceptuando el deposito.
                #Agregamos al filtro las ciudades que involucren sobrepasar el limite de longitud
                sub = sub_touring(ants[k]['Tour'], 0)
                #print("Tour separado: ",sub)
                #print("Costo ultimo sub: ",cost_function(sub[-1]))
                for c in range(1,n_var-1):
                  s = copy.deepcopy(sub[-1])
                  if c not in filter: # Para todas las ciudades que aun no estan filtradas verificamos que puedan ir y llegar al deposito sin superar la capacidad.
                    s.append(c)
                    #s.append(0)
                    cost_s = cost_function.use_function(s, model, "F")
                    #print("Subtour candidato: ",s)
                    #print("Costo asociado: ", cost_s)
                    if L < cost_s: #Si se supera la capacidad, se filtra esta ciudad.
                        filter.append(c)
                #Aplicamos el filtro
                P[filter] = 0
                P = P / np.sum(P)  # Normalización de las probabilidades
                j = roulette_wheel_selection(P)  # Próxima ciudad a visitar
                ants[k]['Tour'].append(j)  # Agregar la ciudad al tour de la hormiga
                if j != 0: #Si la ciudad visitada no es el deposito
                  pending = pending - 1 #Decrementamos las ciudades restantes

            ants[k]['Cost'] = cost_function.use_function(ants[k]['Tour'], model, "F")  # Calcular el costo del tour

            if best_cost[it] > ants[k]['Cost']:
              best_cost[it] = ants[k]['Cost']  # Almacenar el mejor costo de esta iteración
            elif worst_cost[it] < ants[k]['Cost']:
              worst_cost[it] = ants[k]['Cost'] # Almacenar el peor costo de esta iteración

            if ants[k]['Cost'] < best_sol['Cost']:  # Actualizar la mejor solución si es necesario. Este costo pertenece al periodo actual
              best_sol = ants[k].copy()
              it_best = it + 1

            if ants[k]['Cost'] < b_cost: # Actualizar la mejor o peor solución general
              b_cost = ants[k]['Cost']
            elif ants[k]['Cost'] > w_cost:
              w_cost = ants[k]['Cost']

        # Actualiza las feromonas
        sorted_ants = sort_ants(ants)  # Ordena las hormigas por costo

        elit_ants = elitist_ants(sorted_ants, n_best_ants) # Selecciona las mejores hormigas

        for k in range(len(elit_ants)):  #inicia un bucle que itera sobre todas las hormigas consideradas élite.
            tour = elit_ants[k]['Tour'] #se obtiene el tour de la k-ésima hormiga élite. tour será una lista que representa el recorrido de la hormiga.
            #tour.append(tour[0])

            for l in range(n_var):
                i = tour[l] - 1
                j = tour[l + 1] - 1
                tau[i, j] += Q / elit_ants[k]['Cost'] #    Esta es la parte clave de la actualización de feromonas. Se incrementa el valor de feromonas en la ruta de la ciudad i a la ciudad j.
                                                      #    La cantidad en la que se incrementa depende del parámetro Q y del costo del tour de la hormiga élite.

        tau = (1 - rho) * tau  # Evaporación de feromonas

        #best_cost[it] = best_sol['Cost']  # Almacena el mejor costo

        print(f'Iteración {it + 1}: Mejor Costo = {best_cost[it]}')  # Muestra información de la iteración

        plt.figure(num_figs + 1)
        if r == 1:
          graph_name = f'Solución ACO Elitista\nIteración {it + 1}: Mejor Costo = {best_cost[it]}'
        else:
          graph_name = f'Solución ACO Rank\nIteración {it + 1}: Mejor Costo = {best_cost[it]}'
        plot_solution(best_sol['Tour'], model, graph_name)  # Grafica la solución
        if r == 1:
          graph_phe= f'Feromonas ACO Elitista\nIteración {it+1}'
        else:
          graph_phe= f'Feromonas ACO Rank\nIteración {it+1}'
        plot_pheromones(model,tau,graph_phe)

        #acc_i = accuracy(best_cost[it], b_cost,  w_cost, it+1) # Calcula accuracy para la iteración actual
        #acc = np.append(acc, acc_i) # Agrega la accuracy de la iteración actual a la lista

        limit_flag = it == max_it - 1
        time_flag = time.time() - start_time  >= time_limit
        access_flag = cost_function.over_limit()

        current_time = time.time() - start_time
        if not last_model:
          if current_time > times[time_index]:
            model_index+= 1
            time_index+=1
            model_change = True
            if time_index == len(times)-1:
                last_model=True


        plt.pause(0.01)


    # Resultados finales
    plt.figure()
    plt.plot(best_cost[:it+1], linewidth=2)
    plt.xlabel('Iteración')
    plt.ylabel('Mejor Costo')
    if r == 1:
      plt.title('ACO Elitista')
      plot_pheromones(model, tau, 'Feromonas ACO Elitista')
    else:
      plt.title('ACO Rank')
      plot_pheromones(model, tau, 'Feromonas ACO Rank')
    plt.grid(True)


    flag = ''
    if limit_flag:
      flag = 'limit_flag'
    elif time_flag:
      flag = 'time_flag'
    else:
      flag = 'access_flag'

    best_sols.append(best_sol)

    solution = [best_cost[-1], it_best, best_cost[:it+1], worst_cost[:it+1], models_by_iteration, models_by_iteration_flags, flag, best_sols]  # Mejor costo, número de iteración correspondiente, mejores costos de cada iteración y los modelos utilizados en cada iteración
    plt.pause(1)

    return solution  # Devuelve la solución


In [ ]:
import numpy as np
import math

def ant_colony_optimization(timed_models, max_it, n_ant, Q, alpha, beta, rho, tau0, change_prob, p_elitist_ants, L, method):


    # Se define el problema
    cost_function = ObjFunction(lambda tour, model: tour_length(tour, model))

    # ACO
    if method == 'Simple':
      simple_sol, simple_it_needed, simple_best_costs, simple_worst_costs, simple_models_by_iteration, simple_models_by_iteration_flags, simple_flag, simple_best_sols = simple_aco(timed_models, cost_function, max_it, n_ant, Q, tau0, alpha, beta, rho, L)
      simple_area = get_area(simple_best_costs)
      print(f'Solucion Simple: {simple_sol}, Iteraciones necesarias: {simple_it_needed}, Área bajo la curva: {simple_area}')
      simple_periods = get_periods(simple_models_by_iteration, simple_models_by_iteration_flags, simple_best_costs)
      #Obtener periodos
      simple_models = timed_models[0][:len(simple_periods)]
      get_worst_costs_by_periods(simple_models, simple_periods)
      view_periods(simple_periods)
      simple = {
        'models': simple_models,
        'method': 'Simple',
        'periods': simple_periods,
        'best_costs': simple_best_costs,
        'worst_costs': simple_worst_costs,
        'area': simple_area,
        'flag': simple_flag,
        'n_ant': n_ant
      }


      #Mostramos por pantalla la solucion encontrada
      print('\n')
      for s in simple_best_sols:
        print(f'Solucion Simple: {s}')
        sub = sub_touring(s['Tour'][0:-1], 0)
        for i in range(0,len(sub)):
            print(f'Tour {i+1}: {sub[i]+[0]}')

      return simple

    elif method == 'Elitist':
      elitist_sol, elitist_it_needed, elitist_best_costs, elitist_worst_costs, elitist_models_by_iteration, elitist_models_by_iteration_flags, elitist_flag, elitist_best_sols = elitist_aco(timed_models, cost_function, max_it, n_ant, Q, tau0, alpha, beta, rho, 1, L) #Una unica hormiga deposita feromonas
      elitist_area = get_area(elitist_best_costs)
      print(f'Solucion Elitista: {elitist_sol}, Iteraciones necesarias: {elitist_it_needed}, Área bajo la curva: {elitist_area}')
      elitist_periods = get_periods(elitist_models_by_iteration, elitist_models_by_iteration_flags, elitist_best_costs)
      elitist_models = timed_models[0][:len(elitist_periods)]
      get_worst_costs_by_periods(elitist_models, elitist_periods)
      view_periods(elitist_periods)
      elitist = {
          'models': elitist_models,
          'method': 'Elitista',
          'periods': elitist_periods,
          'best_costs': elitist_best_costs,
          'worst_costs': elitist_worst_costs,
          'area': elitist_area,
          'flag': elitist_flag,
          'n_ant': n_ant
      }


      #Mostramos por pantalla la solucion encontrada
      #print('\n')
      #for e in elitist_best_sols:
        #print(f'Solucion Elitista: {e}')
        #sub = sub_touring(e['Tour'][0:-1], 0)
        #for i in range(0,len(sub)):
            #print(f'Tour {i+1}: {sub[i]+[0]}')

      return elitist

    elif method == 'Rank':
      r = math.floor(0.5 * n_ant)
      rank_sol, rank_it_needed, rank_best_costs, rank_worst_costs, rank_models_by_iteration, rank_models_by_iteration_flags, rank_flag, rank_best_sols = elitist_aco(timed_models, cost_function, max_it, n_ant, Q, tau0, alpha, beta, rho, r, L) #"r" hormigas depositan feromonas
      rank_area = get_area(rank_best_costs)
      print(f'Solucion Rank: {rank_sol}, Iteraciones necesarias: {rank_it_needed}, Área bajo la curva: {rank_area}')
      rank_periods = get_periods(rank_models_by_iteration, rank_models_by_iteration_flags, rank_best_costs)
      rank_models = timed_models[0][:len(rank_periods)]
      get_worst_costs_by_periods(rank_models, rank_periods)
      view_periods(rank_periods)
      rank = {
          'models': rank_models,
          'method': 'Rank',
          'periods': rank_periods,
          'best_costs': rank_best_costs,
          'worst_costs': rank_worst_costs,
          'area': rank_area,
          'flag': rank_flag,
          'n_ant': n_ant
      }

      #Mostramos por pantalla la solucion encontrada
      #print('\n')
      #for r in rank_best_sols:
        #print(f'Solucion Rank: {r}')
        #sub = sub_touring(r['Tour'][0:-1], 0)
        #for i in range(0,len(sub)):
            #print(f'Tour {i+1}: {sub[i]+[0]}')

      return rank


TABLAS

In [ ]:
#Manejo de CSV y DF

import pandas as pd

# libraries for the files in google drive
from pydrive.auth import GoogleAuth
from google.colab import drive, files
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.auth import default
from oauth2client.client import GoogleCredentials
try:
  import gspread
except ModuleNotFoundError:
  if 'google.colab' in str(get_ipython()):
   %pip install gspread
  import gspread

# Función: Obtener una hoja determinada del google sheets 'DynamicAco2024'

# Input:
#   title: Título de la hoja de google sheets a obtener

# Output: Hoja de google sheets solicitada

def get_sheet(title):
  auth.authenticate_user()
  creds, _ = default()
  gc = gspread.authorize(creds)
  worksheet = gc.open("DynamicAco2024")
  sheet1 = worksheet.worksheet(title)
  return sheet1

# Función: Agregar los datos obtenidos en google sheets
#
# Input:
#   ws: Hoja donde se guarda la informacion

def save_data(ws, dataframe):

  ws_ex_columns = ['Ejecucion', 'Metodo', 'Modelo', 'Periodo', 'Iteracion', 'Mejor costo', 'Peor costo', 'Cantidad de hormigas']
  ws_heur_columns = ['Modelo', 'Periodo', 'Peor costo por heuristica']
  ws_models_columns = ['Ejecucion', 'Metodo', 'Modelo', 'Motivo de corte', 'Area bajo la curva', 'Cantidad de hormigas']
  ws_periods_columns = ['Metodo', 'Modelo', 'Periodo', 'Cantidad de hormigas', 'Accuracy promedio', 'DE Accuracy', 'Mejor accuracy promedio', 'DE Mejor Accuracy', 'Stability promedio', 'DE Stability', 'Reactivity promedio', 'DE Reactivity', 'Ejecuciones sin reactivity', 'FAS promedio', 'DE FAS']
  ws_mesures_columns = ['Ejecucion', 'Metodo', 'Modelo', 'Periodo', 'Cantidad de hormigas', 'Accuracy promedio', 'DE Accuracy', 'Mejor accuracy', 'Stability', 'Reactivity', 'FAS']
  ws_fit_deg_columns = ['Modelo', 'Metodo', 'Cantidad de hormigas', 'Fitness Degradation', 'Correlacion']


  if ws.title == 'Ejecuciones':
    ws.append_rows(dataframe[ws_ex_columns].fillna(-1).values.tolist())
  elif ws.title == 'Heurística':
    heu_df = dataframe.drop_duplicates(subset=['Metodo', 'Periodo'], keep='first')
    ws.append_rows(heu_df[ws_heur_columns].fillna(-1).values.tolist())
  elif ws.title == 'Modelos':
    models_df = dataframe.drop_duplicates(subset=['Ejecucion', 'Modelo'], keep='first')
    ws.append_rows(models_df[ws_models_columns].fillna(-1).values.tolist())
  elif ws.title == 'Periodos':
    ws.append_rows(dataframe[ws_periods_columns].fillna(-1).values.tolist())
  elif ws.title == 'Métricas':
    ws.append_rows(dataframe[ws_mesures_columns].fillna(-1).values.tolist())
  elif ws.title == 'Fitness Degradation':
    ws.append_rows(dataframe[ws_fit_deg_columns].fillna(-1).values.tolist())



In [ ]:
# Función: Escribir los headers en la primer fila de la hoja de google sheets

# Input:
#   ws: Hoja de google sheets

# Output:
#   La hoja ws tendrá ahora los headers escritos

ex_headers = ['Ejecución', 'Método', 'Modelo', 'Periodo', 'Iteración', 'Mejor costo', 'Peor costo', 'Cantidad de hormigas', 'Accuracy']
heur_headers = ['Modelo', 'Periodo', 'Peor costo por heurística']
models_headers = ['Ejecución', 'Método', 'Modelo', 'Motivo de corte', 'Área bajo la curva', 'Cantidad de hormigas']
periods_headers = ['Método', 'Modelo', 'Periodo', 'Cantidad de hormigas', 'Accuracy promedio', 'DE Accuracy', 'Mejor accuracy promedio', 'DE Mejor Accuracy', 'Stability promedio', 'DE Stability', 'Reactivity promedio', 'DE Reactivity', 'Ejecuciones sin reactivity', 'FAS promedio', 'DE FAS']
measures_headers = ['Ejecución', 'Método', 'Modelo', 'Periodo', 'Cantidad de hormigas', 'Accuracy promedio', 'DE Accuracy', 'Mejor accuracy', 'Stability', 'Reactivity', 'FAS']
fit_deg_headers = ['Modelo', 'Método', 'Cantidad de hormigas', 'Fitness Degradation', 'Correlación']

def write_headers(ws):

  if ws.title == 'Ejecuciones':
    ws.append_row(ex_headers)
  elif ws.title == 'Heurística':
    ws.append_row(heur_headers)
  elif ws.title == 'Modelos':
    ws.append_row(models_headers)
  elif ws.title == 'Periodos':
    ws.append_row(periods_headers)
  elif ws.title == 'Métricas':
    ws.append_row(measures_headers)
  elif ws.title == 'Fitness Degradation':
    ws.append_row(fit_deg_headers)



In [ ]:
import pandas as pd

# Función: Leer parámetros para la ejecución del algoritmo desde google sheets

# Input:
#   ws: Hoja de google sheets con los parámetros

# Output: Data frame de pandas con los nombres de los parámetros como columnas y los valores como filas

def read_parameters(ws):
  return pd.DataFrame(ws.get_all_records())

In [ ]:
# Función: Obtener los parámetros para la ejecución del algoritmo

def get_parameters():

  param_ws = get_sheet('Parámetros')
  param = read_parameters(param_ws) # Leer parámetros desde google sheets

  max_eval = param['Máximas evaluaciones'][0]
  n_ant = param['Cantidad de hormigas'][0]
  Q = param['Q'][0]
  alpha = param['Alfa'][0]
  beta = param['Beta'][0]
  rho = param['Rho'][0]
  change_prob = param['Probabilidad de cambio'][0]
  p_elitist_ants = param['Porcentaje hormigas elitistas'][0]

  return max_eval, n_ant, Q, alpha, beta, rho, change_prob, p_elitist_ants

In [ ]:
import numpy as np
import json
import jsonpickle
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/VRP2024/Modelos/'

# Funcion: Guardar los modelos

def save_models(max_it, change_prob):
  data_size = 5
  seq_len = 50
  seed = 10
  time_limit = seq_len * 6
  ##Obtener ultimo id generado
  last_id = 10
  print("Inicio")
  timed_models = create_models_timed(data_size, seq_len, seed, max_it, change_prob, time_limit) # Crear modelos
  for m in timed_models:
    last_id = last_id + 1
    mod = m[0]
    times = m[1]
    time_limit = m[2]
    print(mod, time)
    mod = jsonpickle.encode(mod.tolist())
    model = {'id': last_id,
           'model': mod,
           'times': times.tolist(),
            'time_limit': time_limit}
    print(id)
    file_name = path+str(last_id)+'.json'
    print(file_name)
    with open(file_name, 'w') as f:
      json.dump(model, f)



# Función: Cargar los modelos

def load_models(id_list):
  timed_models = []
  path = '/content/drive/MyDrive/VRP2024/Modelos/'

  #Agregar modelos
  for id in id_list:
    #Buscar modelo
    file_name = path+str(id)+'.json'
    with open(file_name) as f:
      model = json.load(f)
    recreated_model = jsonpickle.decode(model['model'])
    #Agregar modelo a la lista
    timed_models.append([np.asarray(list(recreated_model)), model['times'], model['time_limit']])
  return timed_models

Mounted at /content/drive


In [ ]:
import numpy as np

# Función: Obtener 'la peor' solución dado un modelo.

def get_worst_cost(model):
  tour = [0] # Inicia en el nodo 0
  cost = 0
  dist = model.get_dist()

  for i in range(1, model.get_n()):
    cost += dist[0][i]*2 #Calcular costo de ida y vuelta
    tour.append(i)
    tour.append(0)

  return cost

In [ ]:
def get_worst_costs_by_periods(models, periods):
  for period, model in zip(periods, models):
    worst_cost = get_worst_cost(model)
    period.set_worst_cost_by_heuristic(worst_cost)

In [ ]:
#Función: Obtiene el mejor costo de cada modelo en cada periodo

#Input:
#   ex_wf: DataFrame con los mejores costos de cada iteración de cada periodo por cada modelo. Columnas: [Modelo, Periodo, Mejor costo]

#Output: Series con clave (modelo, periodo) y valor mejor (minimo) costo

def get_best_cost_by_model_and_period(ex_df):
  return ex_df.groupby(['Modelo', 'Cantidad de hormigas', 'Periodo'])['Mejor costo'].min()

In [ ]:
# Función: Armar dataframe con los datos de las ejecuciones

#Input:
#   aco: Los datos de la solucion que se quieren guardar
#   model_number: Número del modelo asociado
#   ex_number: Numero de la ejecucion asociada

#Output: Dataframe armado

flags = { # Correspondencias de las flags
    'limit_flag': 'Cantidad de iteraciones',
    'time_flag': 'Tiempo terminado',
    'access_flag': 'Cantidad de accesos a la función'
}
def make_dataframe(aco, model_number, ex_number):
  global flags

  df = pd.DataFrame(columns=['Ejecucion', 'Metodo', 'Motivo de corte', 'Area bajo la curva', 'Modelo', 'Periodo', 'Iteracion', 'Mejor costo', 'Peor costo', 'Peor costo por heuristica', 'Cantidad de hormigas'])

  for period_number, period in enumerate(aco['periods']):
    for it in period.get_it(): # Escribir información sobre cada una de las iteraciones pertenecientes al periodo
        best_cost = aco['best_costs'][it] # Calcular mejor costo
        worst_cost = aco['worst_costs'][it] # Calcular peor costo
        df.loc[len(df)] = [ex_number, aco['method'], flags[aco['flag']], aco['area'], model_number, period_number+1, it+1, best_cost, worst_cost, period.get_worst_cost_by_heuristic(), aco['n_ant']]

  return df

In [ ]:
# Función: Calcula accuracy para cada iteración de cada periodo de cada modelo

# Input:
#   executions_df: DataFrame con los mejores costos de cada iteración de cada periodo por cada modelo. Columnas: [Modelo, Periodo, Iteración, Mejor costo]
#   best_costs: Series con los datos del mejor costo por periodo de cada modelo. Indexado (modelo, cantidad de hormigas, periodo)
#   worst_costs: DataFrame con los datos del peor costo por cada periodo de cada modelo. Columnas: [Modelo, Periodo, Peor costo por heurística]

# Output: Dataframe con la accuracy para cada iteración de cada periodo de cada modelo

def get_accuracy_by_periods(executions_df, best_costs, worst_costs):
  accuracy_df = pd.DataFrame(columns=['Periodo', 'Iteracion', 'Accuracy'])

  for index, row in executions_df.iterrows():
    model = row['Modelo']
    period = row['Periodo']
    n_ants = row['Cantidad de hormigas']
    it = row['Iteración']
    it_cost = row['Mejor costo']
    b_cost = best_costs[model, n_ants, period]
    w_cost = worst_costs[(worst_costs['Modelo'] == model) & (worst_costs['Periodo'] == period)]['Peor costo por heurística'].values[0]
    acc = accuracy(it_cost, b_cost, w_cost)
    accuracy_df.loc[len(accuracy_df)] = [period, it, acc]

  return accuracy_df

In [ ]:
def save_accuracy(ws, accuracy):
  acc = accuracy['Accuracy']
  last_row = len(acc.index) + 1
  col = [[val] for val in acc.to_list()]
  ws.update(f'I2:I{last_row}', col)

In [ ]:
def periods_data(period):
  mean_accuracy = period['Accuracy promedio'].mean()
  std_accuracy = period['Accuracy promedio'].std()
  mean_best_accuracy = period['Mejor accuracy'].mean()
  std_best_accuracy = period['Mejor accuracy'].std()

  rows_wo_reactivity = period[period['Reactivity'].isin(['-', 'inf'])]
  count_ex_wo_react = rows_wo_reactivity.shape[0]
  mean_reactivity = period.drop(rows_wo_reactivity.index)['Reactivity'].mean()
  std_reactivity = period.drop(rows_wo_reactivity.index)['Reactivity'].std()

  mean_stability = period[~period['Stability'].isin(['-'])]['Stability'].mean()
  std_stability = period[~period['Stability'].isin(['-'])]['Stability'].std()

  mean_fas = period['FAS'].mean()
  std_fas = period['FAS'].std()

  return pd.Series({'Accuracy promedio': mean_accuracy,
                    'DE Accuracy': std_accuracy,
                   'Mejor accuracy promedio': mean_best_accuracy,
                    'DE Mejor Accuracy': std_best_accuracy,
                    'Stability promedio': mean_stability,
                    'DE Stability': std_stability,
                    'Reactivity promedio': mean_reactivity,
                    'DE Reactivity': std_reactivity,
                    'Ejecuciones sin reactivity': count_ex_wo_react,
                   'FAS promedio': mean_fas,
                    'DE FAS': std_fas})


In [ ]:
def make_periods_dataframe(metrics_df):
  metrics_df = metrics_df.rename(columns={'Ejecución': 'Ejecucion', 'Método':'Metodo'})

  groups = metrics_df.groupby(['Metodo', 'Modelo', 'Periodo', 'Cantidad de hormigas'])
  df = groups.apply(periods_data).reset_index()

  df = df.sort_values(by=['Metodo', 'Modelo', 'Periodo'], ascending=[False,True,True])

  return df

In [ ]:
def metrics_data(group):
  mean_accuracy = group['Accuracy'].mean()
  std_accuracy = group['Accuracy'].std()
  best_accuracy = group['Accuracy'].max()
  stab = stability(group)
  react = reactivity(group)
  fas = fitness_adaptation_speed(group)

  return pd.Series({'Accuracy promedio': mean_accuracy,
                   'DE Accuracy': std_accuracy,
                   'Mejor accuracy': best_accuracy,
                   'Stability': stab,
                   'Reactivity': react,
                   'FAS': fas}
  )

In [ ]:
def make_metrics_dataframe(executions_df):
  executions_df = executions_df.rename(columns={'Ejecución': 'Ejecucion', 'Método':'Metodo', 'Iteración': 'Iteracion'})

  df = executions_df.groupby(['Ejecucion', 'Metodo', 'Modelo', 'Periodo', 'Cantidad de hormigas']).apply(metrics_data).reset_index()
  df.loc[df['Periodo'] == 1, 'Reactivity'] = '-' #No se analiza reactivity para el primer periodo, pq no hubieron cambios en el entorno
  df.loc[df['Reactivity'] == -1, 'Reactivity'] = 'inf'
  df.loc[df['Periodo'] == 1, 'Stability'] = '-' #No se analiza stability para el primer periodo, pq no hubieron cambios en el entorno
  df = df.sort_values(by=['Metodo', 'Modelo','Periodo'], ascending=[False,True,True])
  return df

In [ ]:
def get_fitness_degradation(periods_df):
  df = periods_df.groupby(['Metodo', 'Modelo', 'Cantidad de hormigas']).apply(fitness_degradation).reset_index()
  df.sort_values(by=['Metodo', 'Modelo'], ascending=[False,True])
  return df

In [ ]:
import numpy as np
import math
# Función: Ejecutar el algoritmo n veces

def run():

  ex_ws = get_sheet('Ejecuciones') # Obtener hoja de ejecuciones
  #ex_ws.clear() # Borrar valores de la hoja de ejecuciones
  #write_headers(ex_ws) # Escribir headers en la hoja de ejecuciones

  heur_ws = get_sheet('Heurística')
  #heur_ws.clear()
  #write_headers(heur_ws)

  models_ws = get_sheet('Modelos')
  #models_ws.clear()
  #write_headers(models_ws)

  period_ws = get_sheet('Periodos')
  #period_ws.clear()
  #write_headers(period_ws)

  metrics_ws = get_sheet('Métricas')
  #metrics_ws.clear()
  #write_headers(metrics_ws)

  fit_deg_ws = get_sheet('Fitness Degradation')
  #fit_deg_ws.clear()
  #write_headers(fit_deg_ws)


  max_eval, n_ant, Q, alpha, beta, rho, change_prob, p_elitist_ants = get_parameters() # Obtener parámetros

  #Parametros extra
  L = 1000
  #save_models(max_eval, change_prob)
  id_list = [(13, 'B')]#[(3,'A'),(3,'B'),(6, 'A'), (6, 'B'), (13, 'A'), (13, 'B')] #Número de modelo y criterio para cantidad de hormigas

  timed_models = load_models([m[0] for m in id_list])
  #timed_models = create_models_timed(1, 8, 10, max_it, change_prob, 3) # Crear modelos



  executions = 5 # Número de ejecuciones

  for i, model in enumerate(timed_models):

    model_number = id_list[i][0]
    if id_list[i][1] == 'A':
      n_ant = int(math.floor(model[0][0].get_n() * 0.6))
    elif id_list[i][1] == 'B':
      n_ant = int(math.floor(10+2*(model[0][0].get_n() ** (1/2))))

    for i in range(executions): # Realizar ejecuciones

      tau0 = 10*Q/(model[0][0].get_n()*np.mean(model[0][0].get_dist()))
      simple = ant_colony_optimization(model, 9999, n_ant, Q, alpha, beta, rho, tau0, change_prob, p_elitist_ants, L, 'Simple') #Reemplazamos max_it por 9999 para que siempre termine por tiempo.
      elitist = ant_colony_optimization(model, 9999, n_ant, Q, alpha, beta, rho, tau0, change_prob, p_elitist_ants, L, 'Elitist') #Reemplazamos max_it por 9999 para que siempre termine por tiempo.
      rank = ant_colony_optimization(model, 9999, n_ant, Q, alpha, beta, rho, tau0, change_prob, p_elitist_ants, L, 'Rank') #Reemplazamos max_it por 9999 para que siempre termine por tiempo.

      # Escribir los datos de cada método ejecutado
      global simple_df
      simple_df = make_dataframe(simple, model_number, i + 1)
      elitist_df = make_dataframe(elitist, model_number, i + 1)
      rank_df = make_dataframe(rank, model_number, i + 1)

      #Guardar ejecuciones
      save_data(ex_ws, simple_df)
      save_data(ex_ws, elitist_df)
      save_data(ex_ws, rank_df)

      #Guardar modelos
      save_data(models_ws, simple_df)
      save_data(models_ws, elitist_df)
      save_data(models_ws, rank_df)


    #Guardar heurística
    save_data(heur_ws, simple_df)
run()

In [ ]:
def get_metrics():

  ex_ws = get_sheet('Ejecuciones') # Obtener hoja de ejecuciones
  ex_df = pd.DataFrame(ex_ws.get_all_records())

  metrics_ws = get_sheet('Métricas')
  period_ws = get_sheet('Periodos')
  fit_deg_ws = get_sheet('Fitness Degradation')

  worst_costs = pd.DataFrame(get_sheet('Heurística').get_all_records())
  best_costs = get_best_cost_by_model_and_period(ex_df)
  accuracy = get_accuracy_by_periods(ex_df, best_costs, worst_costs)
  save_accuracy(ex_ws, accuracy)


  #Calcular y guardar reactivity y stability
  ex_df = pd.DataFrame(ex_ws.get_all_records())
  metrics_df = make_metrics_dataframe(ex_df)
  save_data(metrics_ws, metrics_df)
  #Calcular y guardar DataFrame con las métricas de cada periodo
  period_df = make_periods_dataframe(metrics_df)
  save_data(period_ws, period_df)

  #Calcular y guardar fitness degradation
  fit_deg_df = get_fitness_degradation(period_df)
  save_data(fit_deg_ws, fit_deg_df)

get_metrics()
